![Linux Academy](la-logo.png)
<hr/>

<center><h1>TensorFlow/Keras Basic Classification</h1></center>

![Sorting Lego bricks](./lego.jpg)

<center><h2>Lego Brick Sorting</h2></center>

# Introduction

In this Linux Academy Hands-on Lab we will be taking a introductory look at __TensorFlow__ (with __Keras__) and use it to make a simple artificial neural network.

## TensorFlow

TensorFlow (backed by Google) is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML and developers easily build and deploy ML powered applications.

_(Source: https://www.tensorflow.org/)_

## Keras

Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

_(Source: https://keras.io/)_

___Or, to put it another way:___

TensorFlow is a complex tool.  Keras has been built ontop of TensorFlow as a more user-friendly interface.  It helps us to rapidly prototype models, and we will be using it in this lab.


# Scenario

We have bricks, lots of brick, Lego bricks that is.  And we need to get them sorted.

We have a collection of photos of different Lego bricks from different angles.  We have 600 photos (no, really we do) and they are all labeled with the name of the brick.

Each photo has been processed, including: increasing the contrast, sharpening, removing the color, inverting the colors, and reducing in size.

|![Sample Lego brick photo](./sample-before.png)|![Sample Lego brick photo](./sample-after.png)|
|----------------------------------------|----------------------------------------|
| Sample before processing                | Sample after processing                |

In addition to this, we have loaded all the images into a single data array for easier loading into algorithm.  _If you're interested in how these images were collected and processed contact me through the Linux Academy Community Slack:_ ```@mike chambers```

We're going to create a simple, deep learning, neural network classifier model.  We will train the model using the photo data and see if it will correctly predict (or infer) the type of a brick from a supplied test image.

# How to Use This Lab

This is a follow along lab.  That is to say that the code in this Jupyter Notebook should be complete, and you could simply execute the notebook and get a result.  However, watch along with the video to learn more about what is happening in the code, and then take the time to experiment with the code, make changes, break it, fix it, learn!

# 1) The Libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

tf.logging.set_verbosity(tf.logging.ERROR)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

# 2) Load the data [numpy]

We have a dataset created from a set of photos of Lego bricks.  In total we have 4 arrays of data that have been saved to files as NumPy arrays.

1. lego-simple-train-images.npy - _Training images, around 80% of the data collected._
2. lego-simple-train-labels.npy - _A list of (integer) labels that identifiy the classes of the training images._
3. lego-simple-test-images.npy  - _Testing images, around 20% of the data collected._
4. lego-simple-test-labels.npy  - _A list of (integer) labels that identifiy the classes of the testing images._ 

First we load the data into runtime arrays:

In [ ]:
train_images = np.load('lego-simple-train-images.npy')
train_labels = np.load('lego-simple-train-labels.npy')
test_images = np.load('lego-simple-test-images.npy')
test_labels = np.load('lego-simple-test-labels.npy')

The label data we have loaded are integer values (1,2,3) lets get some human names for the data classes we're working with:

In [ ]:
# For humans:
class_names = ['2x3 Brick', '2x2 Brick', '1x3 Brick', '2x1 Brick', '1x1 Brick', '2x2 Macaroni', '2x2 Curved End', 'Cog 16 Tooth', '1x2 Handles', '1x2 Grill']

# Or the real Lego codes:
# class_names = ['3002', '3003', '3622', '3004', '3005', '3063', '47457', '94925', '3839a', '2412b']

The data is loaded but it's nice to be able to see it.  Let's take a look at one of the images loaded with the data:

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.xlabel(class_names[train_labels[0]])
plt.show()

Lets look at some more of the data, and make the formating a little nicer:

In [ ]:
plt.figure(figsize=(15,15))
for i in range(20):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

# 3) Training [TensorFlow Keras]

Now the data is loaded, lets start training.  First we need to create a model. We are creating an artificail neural network.  It will have 3 layers:

1. The input layer with enough nodes for our image data
2. A hidden layer with 128 nodes
3. An output player with 10 nodes, one for each of the classes we want to identify

Each layer is densly connected, meaning tht each neuron in one layer is connected to ever neuron in the next layer.

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(48, 48)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

Now we need to complie the model:

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Finnaly its time to train the model.  Note the 'epochs', and watch the accuracy as the it processes:

In [ ]:
history = model.fit(train_images, train_labels, epochs=4)

You can see from the output above that with each epoch the model gets a greater accuracy (acc) score.  When we trained the model we stored the ```history``` of the training in a variable so we can draw a graph of how the training process progressed.  We can plot accuracy and loss over the epochs:

In [ ]:
plt.figure(figsize=(15, 4))

plt.subplot(1, 2, 1)
# Plot training accuracy values
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(1, 2, 2)
# Plot training loss values
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 4) Testing and Analysis [scikit-learn]

Accuracy while training is one thing, let's calculate accuracy against the __testing data__.  If the accuracy achived during training is much greater than what we calculate fot the testing data the model is (probably) over fit, meaning it works well to classify training data, but not testing data:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# 5) Try some predictions:

Let's use our model to make some predictions using our testing data.  Remember the model hasn't 'seen' our testing data, not during training anyway, so when we make predictions using testing images its doing real work:

Here are a couple of functions to help with the display of the prediction data:

In [ ]:
# Function to display the image:

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img, cmap=plt.cm.binary)
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'
  # Print a label with 'predicted class', 'probability %', 'actual class'
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# Function to display the prediction results in a graph:
def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.xticks(range(10))
  plt.yticks([])
  plot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  plot[predicted_label].set_color('red')
  plot[true_label].set_color('green')

## Single Prediction

Lets grab one of the images from the test dataset.

In [ ]:
test_image_number = 8

img = test_images[test_image_number]
plt.figure()
plt.xticks([])
plt.yticks([])
plt.imshow(img, cmap=plt.cm.binary)
plt.show()

We need to do a quick transformation of the image data as the ```predict``` method is expecting a list of images, so we make a list of one:

In [ ]:
img = (np.expand_dims(img,0))
img.shape

Then we pass our image to the ```predict``` method.  The result is a list of probabilities that the image is a certain class:

In [ ]:
predictions_single = model.predict(img)
predictions_single

NumPy has a handy function to find the largest value in the list (which saves us having to look :) ). We can use the value it finds, to look-up the label that's been predicted for this image:

In [ ]:
prediction_result = np.argmax(predictions_single[0])
class_names[prediction_result]

Did we get it right?  Lets look up the class name for the test image (i.e. the real class name, not the predicted one, but we hope its the same!)  Did the model get it right?

In [ ]:
class_names[test_labels[test_image_number]]

Now let's plot a bar chart (using the helper function we defined at the start of this section) so we can get a sence of how well the model classified this image:

In [ ]:
plot_value_array(0, predictions_single, [test_labels[test_image_number]])
plt.xticks(range(10), class_names, rotation=45)
plt.show()

## Batch Prediction

Now lets get prediction values for ALL the test images we have:

In [ ]:
predictions = model.predict(test_images)

Finally let's use our helper functions to summarise the first 16 images in our test data.  How did we do?

In [ ]:
num_rows = 8
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(15, 16))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

That is the end of this Linux Academy Hands-on Lab.  Thanks